In [4]:
##libraries -- these is my 'lazy', go-to, libraries. Many libraries are not used. 

import numpy as np
import scipy as sp
from scipy.interpolate import interp1d
from IPython.display import display, Markdown, Latex
import pandas as pd
import math
from astropy.io import ascii
%matplotlib ipympl
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.colorbar as cb
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import cmath
import time


plt.style.use('default')


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12)
matplotlib.rcParams['animation.embed_limit'] = 2**128

plt.rcParams['figure.autolayout'] = False
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['figure.figsize']=(10,10)
plt.rcParams['mathtext.fontset'] = 'dejavuserif'

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : '18'}

matplotlib.rc('font', **font)
import glob

ModuleNotFoundError: No module named 'astropy'

In [9]:
#wavefunction via Crank Nicholson Method

def wavefunction(dx, dt, timesteps, V_initial_1, V_initial_2, left_1, right_1, left_2, right_2):

    x_span = 1 #overall x-distance
    dx = dx # distance step sizes (initial dx = 0.001)

    t_span = 1 
    dt = dt # initial dt = 10e-7

    x_span_arr = np.linspace(0, x_span, int(x_span/dx)) # span of x positions 
    x_span_arr_phi = np.linspace(-3125, 3125,int(x_span/dx))

    M = len(x_span_arr-1) # maximum index value for m

    t_span_arr = np.linspace(0,t_span, int(t_span/dt)) # span of time frames

    N = len(t_span_arr-1) # maximum index value for n

    time_steps = timesteps

    ## Initial Wave Packet

    ### The plane wave that we multiply our stationary wave with to provide motion ###
    ### also defines the initial real and imaginary components of psi ####

    # intitial position, sigma, and wave number

    x0 = 0.25 #initial position of center of wave packet - set to 1.25 for square wave
    k0 = 500 #initial wavenumber
    E = 0.5*(k0 * k0) #~91 Joules
    sigma = np.sqrt(0.001) # width of wave packet

    #define intitial arrays

    move = np.zeros(len(x_span_arr), dtype=np.complex_) #array to store propagation component of wave
    psi = np.zeros((time_steps, int(len(x_span_arr))),dtype=np.complex_) #array to store wave
    im = np.array([]) #imaginary component
    re = np.array([]) #real component

    # defining intial wave packet
    for i  in range(1, len(x_span_arr)-1):
        arg = np.complex(np.cos(k0*x_span_arr[i]), np.sin(k0*x_span_arr[i]))
        im = np.append(im, np.exp(-(x_span_arr[i] - x0)**2/sigma**2)*np.sin(k0*x_span_arr[i]))
        re = np.append(re, np.exp(-(x_span_arr[i] - x0)**2/sigma**2)*np.cos(k0*x_span_arr[i]) )
        move[i] = arg

    psi[0]  = np.exp(-(x_span_arr - x0)**2/(2*sigma**2))*move
#     psi[0]  = np.exp(-(10000)*(16*x_span_arr - x0)**1000/(2*sigma**2))*move  #square wave attempt

    ### define the wave packet

    # prob = np.conjugate(psi[0])*psi[0]
    # prob = np.real(prob)

    ### Setting V(x), A and B, and the auxiliary function "ee"

    V = np.zeros(int(len(x_span_arr))) #array to store potential
    B = np.zeros(int(len(x_span_arr))) #array to store part of Cayley form
    A = np.complex(0,4*(dx*dx)/dt) # array to store another part of Cayley form

    # Value of the potential barrier/well in units of E
    #V0 = V_initial
    

    #Constructing the potential layout: Use sin(i/50)*E between 0.314 and 0.628
    for i in range(int(len(x_span_arr)*(left_1)),int(len(x_span_arr)*(right_1))):
        V[i] = V_initial_1*E


    #### second potential if wanted ####
    for i in range(int(len(x_span_arr)*left_2),int(len(x_span_arr)*right_2)):
        V[i] = V_initial_2 * E

    #Variable in the wave function that depends on the potential:
    for i in range(0,int(len(x_span_arr))):
        B[i] = 2*(dx*dx)*V[i]+2


    #Auxiliary Function e:
    ee_initial = np.array([B[1]-A])

    ee= np.zeros(int(len(x_span_arr)),dtype=np.complex_)


    ee[0]= 0+0*1j
    ee[1]= B[1]-A


    for i in range(2,len(x_span_arr)):
        ee[i] = B[i] - A - 1/(ee[i-1])

    ### Finding the initial $\Omega(x,t)$

    #Constructing Omega array:
    OM = np.zeros((time_steps, int(len(x_span_arr))),dtype=np.complex_)

    for i in range(0, len(psi[0])-1):
        OM[0][i] = - psi[0][i+1] + (A+B[i])*psi[0][i] - psi[0][i-1]

    ### Finding  $ \ f(x,t )$ labelled "ff"

    ff = np.zeros((time_steps, int(len(x_span_arr))),dtype=np.complex_)

    ff[0][0] = 0
    ff[0][1] = OM[0][1]

    for i in range(2, len(psi[0])-1):
        ff[0][i] = OM[0][i] + ff[0][i-1]/ee[i-1]

    ## Function that gives us $\psi(x,t)$ 

    # routine that uses OM, ff, ee, A and B to contruct the wave function, psi.
    for j in range(0,time_steps-1):
        for i in range(0,len(x_span_arr)-1):

            if i  == 0 or i == len(x_span_arr)-1:
                psi[j+1][len(x_span_arr)-1 - i] = 0



            elif i == len(x_span_arr)-2:
                psi[j+1][len(x_span_arr)-1 - i] = - (ff[j][len(x_span_arr)-1 - i]/ee[len(x_span_arr)-1 - i])



            else:
                psi[j+1][len(x_span_arr)-1 - i] = (psi[j+1][len(x_span_arr) - i]  - ff[j][len(x_span_arr)-1 - i])/ee[len(x_span_arr)-1 - i]



        for i in range(0,int(len(x_span_arr))-1):
            OM[j+1][i] = - psi[j+1][i+1] + (A+B[i])*psi[j+1][i] - psi[j+1][i-1]

        ff[j+1][0] = 0
        ff[j+1][1] = OM[j+1][1]

        for i in range(2, len(psi[0])-1):
            ff[j+1][i] = OM[j+1][i] + ff[j+1][i-1]/ee[i-1]

          
    

    ## Function that gives us  $\phi(x,t)$

    phi = np.fft.fft(psi)

    ## Probability 

    ###  $|\psi^{*} \psi|$

    # gives array of probability distributions for each time step
    prob_arr = np.real(np.conjugate(psi)*psi)

    ### $|\phi^{*}\phi|$

    prob_phi = np.real(np.conjugate(phi)*phi)

    ## Measure Transmission and Reflection Coefficients

    R = np.array([])
    M = np.array([])
    T = np.array([])
    
    if V_initial_2 == 0:

        for  j in range(0, len(prob_arr)):
            R = np.append(R, np.sum(prob_arr[j][0:int(left_1*len(x_span_arr))])*dx)
            M = np.append(M, np.sum(prob_arr[j][int(left_1*len(x_span_arr)):int(right_1*len(x_span_arr))])*dx)
            T = np.append(T, np.sum(prob_arr[j][int(right_1*len(x_span_arr)):int(len(x_span_arr))])*dx)
            
    else:
        
        for  j in range(0, len(prob_arr)):
            R = np.append(R, np.sum(prob_arr[j][0:int(left_1*len(x_span_arr))])*dx)
            M = np.append(M, np.sum(prob_arr[j][int(left_1*len(x_span_arr)):int(right_2*len(x_span_arr))])*dx)
            T = np.append(T, np.sum(prob_arr[j][int(right_2*len(x_span_arr)):int(len(x_span_arr))])*dx)
        
        
    return dx, dt, R, M, T, x_span_arr, x_span_arr_phi, prob_arr, prob_phi, V, im, re

In [10]:
dx, dt, R, M, T, x_span_arr, x_span_arr_phi, prob_arr, prob_phi, V, im, re = wavefunction(dx= 0.001, 
                                                                                          dt = 10e-7, 
                                                                                          timesteps = 2000, 
                                                                                          V_initial_1 = 1, 
                                                                                          V_initial_2 = 0, 
                                                                                          left_1 = 0.5, 
                                                                                          right_1 = 0.54, 
                                                                                          left_2 = 1, 
                                                                                          right_2 = 1)

In [ ]:
import matplotlib.animation as animation
from matplotlib import animation, rc
from IPython.display import HTML
%matplotlib ipympl

#plt.rcParams['figure.figsize']=(10,6)

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
#plt.tight_layout()

fig , (ax, ax1) = plt.subplots(2, 1, figsize = (15, 8), dpi = 150)
#plt.gcf().subplots_adjust(bottom=0.45)
plt.close()
ax0 = ax.twinx()
ax0.set_ylabel(r'$V_{0}$', color = 'g')
ax0.plot(x_span_arr, V/125000, color = 'g', lw = 1) #potential 
ax0.tick_params(axis='y', colors='g')
ax0.set_ylim(min(V)/125000 - .5,2.5)
ax0.set_yticks([ 0, 1, 2])
ax0.set_yticklabels([0,r'$E$', r'$2E$'])

#factors for resolution factor is 'how many frames to skip each second' and frame number is 'how many frames to show'
#in the case of this project, factor*frame_number = 1100 is ideal. 
factor = 4
frame_number = 1


def animate(i):
    #plt.cla()
    ax.clear()

    
    ax.plot(x_span_arr, prob_arr[factor*i], linewidth = 1)
    ax.set_title('Frame = '+ str(factor*i))    

    ax.text(0.05,2, 'Percent of Initial Wave packet:', color='C0')
    ax.text(0.05,1.8, 'Left of Potential = ' + str(np.round(100 * R[factor*i]/R[0], decimals = 4)))
    ax.text(0.05,1.6, 'Within Potential = ' + str(np.round(100 * M[factor*i]/R[0], decimals = 4)))
    ax.text(0.05,1.4, 'Right of Potential = ' + str(np.round(100 * T[factor*i]/R[0], decimals = 4)))
    ax.set_ylabel(r'$\left| \psi (x) \right|$', fontsize = 14, color = 'C0')
    ax.set_xlabel(r'$x$', fontsize = 14)
    ax.set_ylim(min(V)/125000 - .5,2.5)
    ax.set_yticks([])
    
    
    
    
    
    ax1.clear() 
    ax1.plot(x_span_arr_phi, np.roll(prob_phi[factor*i], 500), linewidth = 1)
    ax1.axvline(np.sqrt(factor*max(V)), ls = '--', lw = .5, label = r'$k_{V_{max}}$')
    if min(V) < 0:
        ax1.axvline(- np.sqrt(np.abs(factor*min(V))), ls = '--', lw = .5, color = 'r', label = r'$k_{V_{min}}$')
    else:
        ax1.axvline(np.sqrt(np.abs(factor*min(V))), ls = '--', lw = .5, color = 'r', label = r'$k_{V_{min}}$')
    ax1.set_ylabel(r'$\left| \phi (k) \right|$', fontsize = 14)
    ax1.set_xlabel(r'$k$', fontsize = 14) 
    ax1.set_ylim(-500, 10000)  
    ax1.set_yticks([])
    ax1.legend()


interval = 1#in seconds     
anim = animation.FuncAnimation(fig,animate,frame_number,interval=interval*25,blit=True)

#plt.show()
#plt.subplots_adjust(hspace = 0.5)

rc('animation', html='jshtml')

anim

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
anim.save('V0_'+str(max(V)/125000)+'_L_'+str(0.4)+'_R_'+str(0.45)+'sigma.mp4')